In [3]:
from pyspark import SparkConf, SparkContext
import csv

def extract_year(date):
    return date.split("-")[0]


def sort_by_year(item):
    # ('Animation', '2016')
    return item[0][1]
    
if __name__ == "__main__":
    conf = SparkConf().setMaster("local").setAppName("IMDB")
    sc = SparkContext(conf=conf)
    
    #Predlog 4
    #Najzastupljeniji žanrovi. Da li postoji razlika po decenijama ili nekim drugim vremenskim periodima
    #Broj filmova u kategoriji po godinama
    
    mymoviedb = sc.textFile("mymoviedb.csv")
    header = mymoviedb.first()
    
    dataWithoutHeader = mymoviedb.filter(lambda line: line != header)
    
    # Extract from each row: year and genres
    data = dataWithoutHeader.mapPartitions(lambda x: csv.reader(x)).map(lambda x: (extract_year(x[0]),x[7]))

    # Split genres and create a key-value pair for each genre and year
    genre_by_year = data.flatMap(lambda x: [(genre.strip(), x[0]) for genre in x[1].split(",")])
    
    # Count the occurrences of each genre by year
    genre_counts_by_year = genre_by_year.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
    
    sorted_genre_counts_by_year = genre_counts_by_year.sortBy(sort_by_year)
    
    output_path = "genre_counts_by_year.txt"
    sorted_genre_counts_by_year.saveAsTextFile(output_path)

    # Stop the Spark context
    sc.stop()
